In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Cargar las neuronas del gonza
rn_1 = joblib.load('ml/neural_network_device_id_1.joblib')
# Cargar los datos del CSV
df = pd.read_csv('csv/device_id_1_2024-11-03_2024-11-20.csv')
df['hour'] = pd.to_datetime(df['hour'])  # Asegúrate de manejar la columna 'hour'


# Calcular la columna 'result' en base a la predicción de humedad
def predict_soil_moisture(row):
    air_temperature = row['avg_air_temperature']
    air_humidity = row['avg_air_humidity']
    soil_humidity = row['avg_soil_humidity']

    new_data = np.array([[air_temperature, air_humidity, soil_humidity]])
    predicted_change = rn_1.predict(new_data)[0][0]
    return round(soil_humidity + predicted_change, 2)


# Aplicar la predicción de la humedad proyectada para cada fila
df['soil_humidity_predicted'] = df.apply(predict_soil_moisture, axis=1)

# Definir los límites del rango óptimo
optimal_range = (40, 58)  # (mínimo, máximo)

# Crear la variable objetivo 'is_optimal' (0 = No es óptimo, 1 = Es óptimo)
df['is_optimal'] = df['soil_humidity_predicted'].between(optimal_range[0], optimal_range[1]).astype(int)

# Seleccionar las variables predictoras y la variable objetivo
X = df[['avg_air_temperature', 'avg_air_humidity', 'soil_humidity_predicted']].values
y = df['is_optimal'].values

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear el modelo de la red neuronal
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # Capa oculta 1
    Dropout(0.2),  # Regularización para evitar sobreajuste
    Dense(32, activation='relu'),  # Capa oculta 2
    Dropout(0.2),
    Dense(1, activation='linear')  # Capa de salida (predicción continua)
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=16,
    verbose=1
)

# Evaluar el modelo
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━

C:\Users\gerojasf\Desktop\projects\capstone\farming-smart-analytics\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 233.7866 - mae: 12.5709 - val_loss: 20.8241 - val_mae: 3.1524
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 174.7083 - mae: 10.3664 - val_loss: 36.2179 - val_mae: 5.4708
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 115.0423 - mae: 8.5864 - val_loss: 29.5323 - val_mae: 4.7861
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 113.4218 - mae: 8.2529 - val_loss: 8.5223 - val_mae: 2.4279
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 60.1764 - mae: 6.3577 - val_loss: 2.2504 - val_mae: 1.3038
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 99.0574 - mae: 7.0230 - val_loss: 3.7945 - val_mae: 1.5651
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 91.5625 - mae: 7.6958 - val_loss: 3.8006 - val_mae: 1.5330
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 91.6996 - mae: 7.4919 - val_loss: 1.8235 - val_mae: 1.1642
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 64.5540 - mae: 6.79